In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 500)

In [2]:
o_status = pd.read_csv('C:/Users/elfek/Desktop/NTT/og/04.order_status.csv')
o_item = pd.read_csv('C:/Users/elfek/Desktop/NTT/og/05.order_items.csv', sep = ";")
customers = pd.read_csv('C:/Users/elfek/Desktop/NTT/clean/02.customers_new.csv', encoding='latin-1', index_col=0)
o_payments = pd.read_csv('C:/Users/elfek/Desktop/NTT/og/06.order_payments.csv', sep = ";")
p_reviews = pd.read_csv('C:/Users/elfek/Desktop/NTT/og/07.product_reviews.csv', sep = ";")
products = pd.read_csv('C:/Users/elfek/Desktop/NTT/clean/products_new.csv',index_col=0)

In [3]:
o_status.nunique() , customers.nunique() #note the order_ids are equal to customers_ids counts 

(order_id                       99441
 customer_id                    99441
 order_status                       8
 ts_order_purchase              88797
 ts_order_approved              50469
 ts_order_delivered_carrier     61548
 ts_order_delivered_customer    75653
 ts_order_estimated_delivery      461
 dtype: int64,
 latest_order_purchase            86642
 customer_id                      99441
 customer_unique_id               96096
 customer_autonomous_community       17
 customer_city                       68
 cac_count                            3
 cc_count                             3
 dtype: int64)

In [4]:
#checking the each data type within each dataset, some changes must take place for data representing dates & price/cost 
#since they shouldnt have an object type 
o_status.info() , p_reviews.info() , o_item.info() , o_payments.info() , customers.info() , products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99450 entries, 0 to 99449
Data columns (total 8 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   order_id                     99441 non-null  object
 1   customer_id                  99450 non-null  object
 2   order_status                 99450 non-null  object
 3   ts_order_purchase            99450 non-null  object
 4   ts_order_approved            99288 non-null  object
 5   ts_order_delivered_carrier   97663 non-null  object
 6   ts_order_delivered_customer  96481 non-null  object
 7   ts_order_estimated_delivery  99446 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100802 entries, 0 to 100801
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   order_id      100802 non-null  object
 1   product_id    100802 non-null  object


(None, None, None, None, None, None)

In [5]:
#Convert from object to datetime for all dates 
o_status['ts_order_purchase'] = pd.to_datetime(o_status['ts_order_purchase'], format='%d/%m/%Y %H:%M')
o_status['ts_order_approved'] = pd.to_datetime(o_status['ts_order_approved'], format='%d/%m/%Y %H:%M')
o_status['ts_order_delivered_carrier'] = pd.to_datetime(o_status['ts_order_delivered_carrier'], format='%d/%m/%Y %H:%M')
o_status['ts_order_delivered_customer'] = pd.to_datetime(o_status['ts_order_delivered_customer'], format='%d/%m/%Y %H:%M')
o_status['ts_order_estimated_delivery'] = pd.to_datetime(o_status['ts_order_estimated_delivery'], format='%d/%m/%Y %H:%M')
p_reviews['review_date'] = pd.to_datetime(p_reviews['review_date'], format='%d/%m/%Y %H:%M')

In [6]:
#Convert from object to float for all dates 
o_item['price'] = [float(str(i).replace(",", ".")) for i in o_item["price"]]
o_item['shipping_cost'] = [float(str(i).replace(",", ".")) for i in o_item["shipping_cost"]]
o_payments['transaction_value'] = [float(str(i).replace(",", ".")) for i in o_payments["transaction_value"]]

---

## Preparing order item dataset

In [7]:
o_item

,order_id,order_item_sequence_id,product_id,price,shipping_cost,seller_id,max_shipping_seller_date
0,e5fa5a7210941f7d56d0208e4e071d35,1,f3c2d01a84c947b078e32bbef0718962,59.50,15.56,a425f92c199eb576938df686728acd20,19/09/2017 00:15
1,bfbd0f9bdef84302105ad712db648a6c,1,5a6b04657a4c5ee34285d1e4619a96b4,44.99,2.83,ecccfa2bb93b34a3bf033cc5d1dcdc69,19/09/2017 23:11
2,bfbd0f9bdef84302105ad712db648a6c,2,5a6b04657a4c5ee34285d1e4619a96b4,44.99,2.83,ecccfa2bb93b34a3bf033cc5d1dcdc69,19/09/2017 23:11
3,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,44.99,2.83,ecccfa2bb93b34a3bf033cc5d1dcdc69,19/09/2017 23:11
4,cd3b8574c82b42fc8129f6d502690c3e,1,e2a1d45a73dc7f5a7f9236b043431b89,29.99,10.96,b499c00f28f4b7069ff6550af8c1348a,08/10/2017 10:34
...,...,...,...,...,...,...,...
112650,3b61aab5de69abc1731138bd104a777f,1,6aa063e063f2ab982b471e58afe06d72,999.99,24.77,610f72e407cdd7caaa2f8167b0163fd8,18/09/2019 21:10
112651,9c94a4ea2f7876660fa6f1b59b69c8e6,1,282b126b2354516c5f400154398f616d,75.99,14.70,7a241947449cc45dbfda4f9d0798d9d0,03/02/2020 20:23
112652,13bdf405f961a6deec817d817f5c6624,1,96ea060e41bdecc64e2de00b97068975,69.99,14.66,7a241947449cc45dbfda4f9d0798d9d0,05/02/2020 03:30
112653,c2bb89b5c1dd978d507284be78a04cb2,1,87b92e06b320e803d334ac23966c80b1,99.99,61.44,7a241947449cc45dbfda4f9d0798d9d0,09/04/2020 22:35


In [8]:
o_item.isna().sum()

order_id                    5
order_item_sequence_id      0
product_id                  0
price                       0
shipping_cost               0
seller_id                   0
max_shipping_seller_date    0
dtype: int64

In [9]:
o_item = o_item.dropna()

In [10]:
#No need to have these two columns as they dont effect the customer segmentation
o_item = o_item.drop(columns=['max_shipping_seller_date', 'seller_id'])

In [11]:
#Here we prep the dataset, we take max of order item id to have the information of how many items were ordered , we join the unique items so that we know if that item is 
#the same or it was order of different items, sum the price and shipping cost
o_item1 = o_item.groupby(['order_id'], as_index = False).agg({'order_item_sequence_id': 'max', 'product_id': lambda x: ','.join(x.unique()), 'price': 'sum', 'shipping_cost': 'sum'})

In [12]:
#we would like to calculate for the transaction value of the order by adding summed price and shipping cost
o_item1['total'] = o_item1.price + o_item1.shipping_cost
o_item1.columns = o_item1.columns.str.replace('shipping_cost', 'tot_shipping_cost')
o_item1.columns = o_item1.columns.str.replace('price', 'tot_price')
o_item1.columns = o_item1.columns.str.replace('product_id', 'product_id_combined')

In [13]:
#here we are counting the unique products ordered for  
o_item2 = o_item1['product_id_combined'].str.count(',').add(1).groupby(o_item1.order_id).sum()
o_item2 = o_item2.to_frame()
o_item2.columns = o_item2.columns.str.replace('product_id_combined', 'product_count')

In [14]:
o_item2.value_counts(normalize=True).round(4)*100

product_count
1                96.72
2                 2.88
3                 0.30
4                 0.07
6                 0.01
5                 0.01
7                 0.00
8                 0.00
dtype: float64

In [15]:
o_item1.columns = o_item1.columns.str.replace('order_item_sequence_id' , 'order_item_count')

In [16]:
o_item3 = pd.merge(o_item1 , o_item2, on = 'order_id', how = 'left')

In [17]:
o_item = pd.merge(o_item[['product_id','order_id']] , o_item3, on = 'order_id', how = 'inner')

In [18]:
o_item

,product_id,order_id,order_item_count,product_id_combined,tot_price,tot_shipping_cost,total,product_count
0,f3c2d01a84c947b078e32bbef0718962,e5fa5a7210941f7d56d0208e4e071d35,1,f3c2d01a84c947b078e32bbef0718962,59.50,15.56,75.06,1
1,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,134.97,8.49,143.46,1
2,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,134.97,8.49,143.46,1
3,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,134.97,8.49,143.46,1
4,e2a1d45a73dc7f5a7f9236b043431b89,cd3b8574c82b42fc8129f6d502690c3e,1,e2a1d45a73dc7f5a7f9236b043431b89,29.99,10.96,40.95,1
...,...,...,...,...,...,...,...,...
112645,6aa063e063f2ab982b471e58afe06d72,3b61aab5de69abc1731138bd104a777f,1,6aa063e063f2ab982b471e58afe06d72,999.99,24.77,1024.76,1
112646,282b126b2354516c5f400154398f616d,9c94a4ea2f7876660fa6f1b59b69c8e6,1,282b126b2354516c5f400154398f616d,75.99,14.70,90.69,1
112647,96ea060e41bdecc64e2de00b97068975,13bdf405f961a6deec817d817f5c6624,1,96ea060e41bdecc64e2de00b97068975,69.99,14.66,84.65,1
112648,87b92e06b320e803d334ac23966c80b1,c2bb89b5c1dd978d507284be78a04cb2,2,87b92e06b320e803d334ac23966c80b1,199.98,122.88,322.86,1


In [19]:
#(o_item['product_count']/o_item['order_item_count']).value_counts()

---

### Preparing order status

In [20]:
o_status_del = o_status[o_status['order_status'] == "delivered"]

In [21]:
del o_status_del['ts_order_delivered_carrier']
del o_status_del['ts_order_approved']

In [22]:
o_status_del.isna().sum() 

order_id                       5
customer_id                    0
order_status                   0
ts_order_purchase              0
ts_order_delivered_customer    8
ts_order_estimated_delivery    0
dtype: int64

In [23]:
o_status_del = o_status_del.dropna()

In [24]:
o_status_del.nunique()

order_id                       96470
customer_id                    96470
order_status                       1
ts_order_purchase              86350
ts_order_delivered_customer    75645
ts_order_estimated_delivery      445
dtype: int64

In [25]:
o_status_del['diff'] = o_status_del['ts_order_estimated_delivery'] - o_status_del['ts_order_purchase']

---

* ***Preparing products & o_item***

In [26]:
products = products.reset_index()

In [27]:
products

,product_id,product_category_name,product_weight_gr,product_length_cm,product_height_cm,product_width_cm,product_photo_quantity,category_new
0,1e9e8ef04dbcff4541ed26657ea517e5,fragrance,225.0,16.0,10.0,14.0,1,beauty & personal care
1,6a2fb4dd53d2cdb88e0432f1284a004c,fragrance,400.0,27.0,5.0,20.0,2,beauty & personal care
2,0d009643171aee696f4733340bc2fdd0,fragrance,422.0,21.0,16.0,18.0,1,beauty & personal care
3,b1eae565a61935e0011ee7682fef9dc9,fragrance,267.0,17.0,13.0,17.0,2,beauty & personal care
4,8da90b37f0fb171b4877c124f965b1f6,fragrance,377.0,18.0,13.0,15.0,3,beauty & personal care
...,...,...,...,...,...,...,...,...
32336,13b25797ad1e6d6c7cd3cbeb35f8ee7a,mattresses & pillows,18250.0,25.0,25.0,25.0,1,home & kitchen
32337,dc36a7859b743d8610a2bbbaea26ece9,mattresses & pillows,2500.0,41.0,21.0,21.0,2,home & kitchen
32338,107fde0930956120d1e13dd1062fbb46,mattresses & pillows,30000.0,60.0,73.0,60.0,1,home & kitchen
32339,726b4e18f00255e2e63491bcba3f60b8,mattresses & pillows,1700.0,70.0,10.0,100.0,1,home & kitchen


In [28]:
products.isna().sum()

product_id                0
product_category_name     0
product_weight_gr         1
product_length_cm         1
product_height_cm         1
product_width_cm          1
product_photo_quantity    0
category_new              0
dtype: int64

In [29]:
products = products[['product_category_name','category_new','product_id']]

In [30]:
o_item_prod = o_item[['product_id','order_id']]

In [31]:
o_product = pd.merge(o_item_prod , products, on='product_id', how='inner')

In [32]:
o_product.nunique()

product_id               32341
order_id                 97277
product_category_name       68
category_new                23
dtype: int64

In [33]:
o_product.isna().sum()

product_id               0
order_id                 0
product_category_name    0
category_new             0
dtype: int64

In [34]:
o_product.columns = o_product.columns.str.replace('product_category_name', 'og_category')
o_product.columns = o_product.columns.str.replace('category_new', 'new_category')

In [35]:
o_product.shape

(111047, 4)

In [36]:
o_product = o_product[o_product.order_id.isin(o_status_del.order_id)]

In [37]:
o_product.shape

(108652, 4)

---

* ***Preparing Customers and order count per customer***

In [38]:
customers.nunique()

latest_order_purchase            86642
customer_id                      99441
customer_unique_id               96096
customer_autonomous_community       17
customer_city                       68
cac_count                            3
cc_count                             3
dtype: int64

In [39]:
del customers['latest_order_purchase']

In [40]:
customers.isna().sum()

customer_id                      0
customer_unique_id               0
customer_autonomous_community    0
customer_city                    0
cac_count                        0
cc_count                         0
dtype: int64

In [41]:
o_customer = o_status_del[['order_id','customer_id']]

In [42]:
customers = pd.merge(customers , o_customer, on = 'customer_id', how='inner')

In [43]:
# we are computing for both in order to CHECK mainly if customer_id and order_id are in line with each other, then 
#if this hypoethesis is proven to be right, we will only selecting one column to add otherwise opt only for order_id
a = customers.groupby('customer_unique_id', as_index=False)['customer_id'].nunique()
b = customers.groupby('customer_unique_id', as_index=False)['order_id'].nunique()

In [44]:
a.customer_id.value_counts() , b.order_id.value_counts()

(1     90549
 2      2573
 3       181
 4        28
 5         9
 6         5
 7         3
 9         1
 15        1
 Name: customer_id, dtype: int64,
 1     90549
 2      2573
 3       181
 4        28
 5         9
 6         5
 7         3
 9         1
 15        1
 Name: order_id, dtype: int64)

In [45]:
a[a['customer_id'] == 7] , b[b['order_id'] == 7]

(                     customer_unique_id  customer_id
 10058  1b6c7548a2a1f9037c1fd3ddfed95f33            7
 36701  6469f99c1f9dfae7733b25662e7f1782            7
 73916  ca77025e7201e3b30c44b472ff346268            7,
                      customer_unique_id  order_id
 10058  1b6c7548a2a1f9037c1fd3ddfed95f33         7
 36701  6469f99c1f9dfae7733b25662e7f1782         7
 73916  ca77025e7201e3b30c44b472ff346268         7)

In [46]:
c = pd.merge(a , b , on = 'customer_unique_id', how='inner')

In [47]:
(c['customer_id']/c['order_id']).value_counts() ##this proves that they are compatible 

1.0    93350
dtype: int64

In [48]:
customers.nunique()

customer_id                      96470
customer_unique_id               93350
customer_autonomous_community       17
customer_city                       68
cac_count                            3
cc_count                             3
order_id                         96470
dtype: int64

In [49]:
a.shape , b.shape , customers.shape

((93350, 2), (93350, 2), (96477, 7))

In [50]:
b.columns = b.columns.str.replace('order_id', 'order_count')

In [51]:
customers = pd.merge(customers, b, on ='customer_unique_id',how="left")

In [52]:
customers.shape

(96477, 8)

In [53]:
customers = customers[customers.order_id.isin(o_status_del.order_id)]

In [54]:
customers.shape

(96477, 8)

---

* ***Preparing o_payments***

In [55]:
o_payments.isna().sum()

order_id                         12
payment_method                    0
payment_method_sequence_id        0
payment_installments_quantity     0
transaction_value                 0
dtype: int64

In [56]:
o_payments = o_payments.dropna() #we just have to drop these missing order_ids

In [57]:
customers_reduced = customers[['order_id', 'customer_unique_id']]

In [58]:
o_payments = pd.merge(customers_reduced , o_payments, on = 'order_id', how="left")

In [59]:
o_payments.isna().sum()

order_id                         0
customer_unique_id               0
payment_method                   1
payment_method_sequence_id       1
payment_installments_quantity    1
transaction_value                1
dtype: int64

In [60]:
o_payments = o_payments.dropna()

In [61]:
o_payments.columns

Index(['order_id', 'customer_unique_id', 'payment_method',
       'payment_method_sequence_id', 'payment_installments_quantity',
       'transaction_value'],
      dtype='object')

In [62]:
o_payments1 = o_payments.groupby(['customer_unique_id'], as_index = False).agg({'payment_installments_quantity': 'max', 'payment_method': lambda x: ','.join(x.unique()), 'transaction_value': 'sum', 'payment_method_sequence_id': 'max'})

In [63]:
o_payments.order_id.nunique() , o_status_del.order_id.nunique()

(96469, 96470)

In [64]:
o_payments1.transaction_value.describe() , o_item.total.describe() #different since o_payments only include delivered 

(count    93349.000000
 mean       165.220861
 std        226.393337
 min          9.590000
 25%         63.050000
 50%        107.780000
 75%        182.590000
 max      13664.080000
 Name: transaction_value, dtype: float64,
 count    112650.000000
 mean        180.244628
 std         272.829911
 min           9.590000
 25%          65.620000
 50%         114.440000
 75%         195.337500
 max       13664.080000
 Name: total, dtype: float64)

In [65]:
o_payments1

,customer_unique_id,payment_installments_quantity,payment_method,transaction_value,payment_method_sequence_id
0,0000366f3b9a7992bf8c76cfdf3221e2,8.0,credit_card,141.90,1.0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,1.0,credit_card,27.19,1.0
2,0000f46a3911fa3c0805444483337064,8.0,credit_card,86.22,1.0
3,0000f6ccb0745a6a4b88665a16c9f078,4.0,credit_card,43.62,1.0
4,0004aac84e0df4da2b147fca70cf8255,6.0,credit_card,196.89,1.0
...,...,...,...,...,...
93344,fffcf5a5ff07b0908bd4e2dbc735a684,10.0,credit_card,2067.42,1.0
93345,fffea47cd6d3cc0a88bd621562a9d061,1.0,credit_card,84.58,1.0
93346,ffff371b4d645b6ecea244b27531430a,1.0,credit_card,112.46,1.0
93347,ffff5962728ec6157033ef9805bacc48,5.0,credit_card,133.69,1.0


---

* ***Preparing the review_count***

In [66]:
p_reviews = pd.read_csv('C:/Users/elfek/Desktop/NTT/og/07.product_reviews.csv', sep = ";")

In [67]:
p_reviews.shape

(100802, 4)

In [68]:
p_reviews = p_reviews[p_reviews.order_id.isin(o_status_del.order_id)]

In [69]:
p_reviews = pd.merge(p_reviews , customers[['order_id','customer_unique_id']], on='order_id', how='inner')

In [70]:
p_reviews.isna().sum()

order_id              0
product_id            0
review_score          0
review_date           0
customer_unique_id    0
dtype: int64

In [71]:
review_count = p_reviews.groupby('customer_unique_id', as_index=False)['review_score'].count()

In [72]:
review_count = review_count.rename({'review_score' : 'review_count'}, axis = 1)

In [73]:
p_reviews = pd.merge(p_reviews, review_count, on='customer_unique_id',how='inner')

In [74]:
p_reviews[p_reviews['review_count'] == 3]

,order_id,product_id,review_score,review_date,customer_unique_id,review_count
266,00b30bb163474583c14db1689259cf4d,3552627a68384dc559f0fd4cce173269,5,21/02/2019 01:43,1373e04979cfa0fb2092909abbd57f25,3
267,481bba01f1c9a6070e8bdcaf7965b1e0,568c923625669145cc9a644856fccd14,1,22/04/2019 16:29,1373e04979cfa0fb2092909abbd57f25,3
268,612d5630822f4796875619b3a6a7b481,2b27cb23faa03f71607fcdd1539387f3,4,21/05/2019 19:22,1373e04979cfa0fb2092909abbd57f25,3
315,00c9d0257381c7e9ec13b4505f25b2cf,958cd4fc82ce0223c757f47bc9d67e23,3,29/04/2019 17:29,297ec5afd18366f5ba27520cc4954151,3
316,eb4bfa31c0d54df31280baa9f95e4e78,57c9362442482cd2451f2eabf468e532,5,20/05/2019 16:27,297ec5afd18366f5ba27520cc4954151,3
...,...,...,...,...,...,...
98596,ffb8f7de8940249a3221252818937ecb,bd0ac51dc93e62c4dbe6ca9d70a9b311,4,03/08/2019 19:24,22e2a6909c8debb64e8560df49a25260,3
98597,ffb8f7de8940249a3221252818937ecb,bd6e8cf9fe4122c385da2bcb9f979d5d,1,03/08/2019 19:24,22e2a6909c8debb64e8560df49a25260,3
98598,ffb9a9cd00c74c11c24aa30b3d78e03b,03bb06cda40712fb8473f7962fb7d198,5,30/03/2018 03:17,d10fb5de0a3fed8a598aa7fd3aa1be14,3
98599,ffb9a9cd00c74c11c24aa30b3d78e03b,3321ad579f19476d0d668f726f8dffec,2,30/03/2018 03:17,d10fb5de0a3fed8a598aa7fd3aa1be14,3


In [75]:
p_reviews.review_count.value_counts()

1     86559
2      9382
3      1752
4       616
5       160
6       126
7        42
9        18
8        16
15       15
13       13
Name: review_count, dtype: int64

---

### Let's Merge everyting
###### Finally, we need to have a dataset with just customer_unique_id 

In [76]:
customers_reduced = customers_reduced[customers_reduced.order_id.isin(o_status_del.order_id)]

---

In [77]:
o_product = pd.merge(o_product, customers_reduced, on='order_id', how='left')

In [78]:
o_product

,product_id,order_id,og_category,new_category,customer_unique_id
0,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,beauty & personal care,beauty & personal care,830d5b7aaa3b6f1e9ad63703bec97d23
1,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,beauty & personal care,beauty & personal care,830d5b7aaa3b6f1e9ad63703bec97d23
2,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,beauty & personal care,beauty & personal care,830d5b7aaa3b6f1e9ad63703bec97d23
3,5a6b04657a4c5ee34285d1e4619a96b4,38c5c2886f2aab75bb1ae3f79a3f300d,beauty & personal care,beauty & personal care,4766fb51c287a3a3a67eae04ed0e75a4
4,e2a1d45a73dc7f5a7f9236b043431b89,cd3b8574c82b42fc8129f6d502690c3e,handbags & accessories,clothing & accessories,87776adb449c551e74c13fc34f036105
...,...,...,...,...,...
108656,cedc314dea647b3337f14e20c6fe1234,dc15c503e5d31267bef467c8c63476c4,comics,books,e5a4f6d7cc11cfd5f32e6e5adde8d127
108657,44406b87e5ac6494cdb0c9dccd3b88d3,e7c290bfc31d7eed478c3d3d2d4d2953,tools home improvement,home & kitchen,9dfcc502727549f99c9f73dbd28b35e9
108658,44406b87e5ac6494cdb0c9dccd3b88d3,e7c290bfc31d7eed478c3d3d2d4d2953,tools home improvement,home & kitchen,9dfcc502727549f99c9f73dbd28b35e9
108659,61919b39651acb61ec24307ed8b9502d,59eaa904b3f0dbde2785ac1b27eccd18,cell phones,computers & phones,7dc9f5b742ea9e6e4cff3ca5b1c8a78b


In [79]:
category_combined_per_order = o_product.groupby(['customer_unique_id'], as_index = False).agg({'og_category': lambda x: ','.join(x.unique()), 'new_category': lambda x: ','.join(x.unique())})

category_combined_per_order.columns = category_combined_per_order.columns.str.replace('og_category', 'og_cat_combined')
category_combined_per_order.columns = category_combined_per_order.columns.str.replace('new_category', 'new_cat_combined')

In [80]:
o_product = pd.merge(o_product, category_combined_per_order, on ='customer_unique_id',how='left')

-----

In [81]:
o_item = pd.merge(o_item, customers_reduced, on='order_id', how='inner')

In [82]:
o_item

,product_id,order_id,order_item_count,product_id_combined,tot_price,tot_shipping_cost,total,product_count,customer_unique_id
0,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,134.97,8.49,143.46,1,830d5b7aaa3b6f1e9ad63703bec97d23
1,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,134.97,8.49,143.46,1,830d5b7aaa3b6f1e9ad63703bec97d23
2,5a6b04657a4c5ee34285d1e4619a96b4,bfbd0f9bdef84302105ad712db648a6c,3,5a6b04657a4c5ee34285d1e4619a96b4,134.97,8.49,143.46,1,830d5b7aaa3b6f1e9ad63703bec97d23
3,e2a1d45a73dc7f5a7f9236b043431b89,cd3b8574c82b42fc8129f6d502690c3e,1,e2a1d45a73dc7f5a7f9236b043431b89,29.99,10.96,40.95,1,87776adb449c551e74c13fc34f036105
4,817e1c2d22418c36386406ccacfa53e8,c3d9e402b6a0fbe2a5f7fc5b41117c38,1,817e1c2d22418c36386406ccacfa53e8,189.00,48.45,237.45,1,9f302d00dd3e18ed3745778184b4f0fe
...,...,...,...,...,...,...,...,...,...
110193,61919b39651acb61ec24307ed8b9502d,59eaa904b3f0dbde2785ac1b27eccd18,1,61919b39651acb61ec24307ed8b9502d,299.00,14.75,313.75,1,7dc9f5b742ea9e6e4cff3ca5b1c8a78b
110194,4c2a4020fcd651812100ebbeac1b2753,1afe384f199748cff7a42c9902065560,1,4c2a4020fcd651812100ebbeac1b2753,599.99,29.18,629.17,1,24a438c52577c3c30ba86532b4166ff8
110195,6aa063e063f2ab982b471e58afe06d72,3b61aab5de69abc1731138bd104a777f,1,6aa063e063f2ab982b471e58afe06d72,999.99,24.77,1024.76,1,101375bf617fd60c9eee42f98d9a73d6
110196,87b92e06b320e803d334ac23966c80b1,c2bb89b5c1dd978d507284be78a04cb2,2,87b92e06b320e803d334ac23966c80b1,199.98,122.88,322.86,1,44b6bbfea26596437062a38c8e6bcec1


In [83]:
o_item = o_item.drop_duplicates(['order_id'],keep= 'last')

In [84]:
o_item.columns

Index(['product_id', 'order_id', 'order_item_count', 'product_id_combined',
       'tot_price', 'tot_shipping_cost', 'total', 'product_count',
       'customer_unique_id'],
      dtype='object')

In [85]:
o_item_per_CUID = o_item.groupby(['customer_unique_id'], as_index = False).agg({'total': 'sum', 'tot_shipping_cost': 'max', 'order_item_count':'sum'})
o_item_per_CUID.columns = o_item_per_CUID.columns.str.replace('total', 'tot_spending')
o_item_per_CUID.columns = o_item_per_CUID.columns.str.replace('tot_shipping_cost', 'max_shipping')
o_item_per_CUID.columns = o_item_per_CUID.columns.str.replace('order_item_count', 'tot_order_item_count')

In [86]:
o_item_per_CUID1 = o_item.groupby(['customer_unique_id'], as_index = False).agg({'tot_shipping_cost': 'sum'})
o_item_per_CUID1.columns = o_item_per_CUID1.columns.str.replace('tot_shipping_cost', 'tot_shipping')

In [87]:
o_item = pd.merge(o_item, o_item_per_CUID, on ='customer_unique_id',how='left')

In [88]:
o_item = pd.merge(o_item, o_item_per_CUID1, on ='customer_unique_id',how='left')

In [89]:
o_item.describe()

,order_item_count,tot_price,tot_shipping_cost,total,product_count,tot_spending,max_shipping,tot_order_item_count,tot_shipping
count,96470.000000,96470.000000,96470.000000,96470.000000,96470.000000,96470.000000,96470.000000,96470.000000,96470.000000
mean,1.142210,137.040001,22.785798,159.825799,1.038540,170.753352,23.176697,1.235265,24.516449
std,0.538824,209.052608,21.559959,218.802007,0.227951,233.437574,22.020023,0.765531,24.569123
min,1.000000,0.850000,0.000000,9.590000,1.000000,9.590000,0.000000,1.000000,0.000000
25%,1.000000,45.900000,13.850000,61.850000,1.000000,64.010000,14.020000,1.000000,14.100000
50%,1.000000,86.500000,17.170000,105.280000,1.000000,111.020000,17.480000,1.000000,17.740000
75%,1.000000,149.900000,24.020000,176.260000,1.000000,188.730000,24.870000,1.000000,26.700000
max,21.000000,13440.000000,1794.960000,13664.080000,8.000000,13664.080000,1794.960000,24.000000,1794.960000


In [90]:
o_item[o_item['tot_order_item_count'] == 24]

,product_id,order_id,order_item_count,product_id_combined,tot_price,tot_shipping_cost,total,product_count,customer_unique_id,tot_spending,max_shipping,tot_order_item_count,tot_shipping
92419,e7cc48a9daff5436f63d3aad9426f28b,cb1f3a44e8b8527e16913306a4d3de2f,6,e7cc48a9daff5436f63d3aad9426f28b,1020.0,182.64,1202.64,1,c8460e4251689ba205045f3ea17884a1,4655.88,182.64,24,575.88
92521,e7cc48a9daff5436f63d3aad9426f28b,3e073b27114fe955e184db124d34a566,6,e7cc48a9daff5436f63d3aad9426f28b,1020.0,27.96,1047.96,1,c8460e4251689ba205045f3ea17884a1,4655.88,182.64,24,575.88
92817,e7cc48a9daff5436f63d3aad9426f28b,03aba68b07658f28f29612641f08d4ba,6,e7cc48a9daff5436f63d3aad9426f28b,1020.0,182.64,1202.64,1,c8460e4251689ba205045f3ea17884a1,4655.88,182.64,24,575.88
92819,e7cc48a9daff5436f63d3aad9426f28b,8e8f8ef29c78c7b9c98cac70dd331ce2,6,e7cc48a9daff5436f63d3aad9426f28b,1020.0,182.64,1202.64,1,c8460e4251689ba205045f3ea17884a1,4655.88,182.64,24,575.88


---

In [91]:
o_status_del = pd.merge(o_status_del, customers_reduced, on='order_id', how='inner')

In [92]:
o_status_del.isna().sum()

order_id                       0
customer_id                    0
order_status                   0
ts_order_purchase              0
ts_order_delivered_customer    0
ts_order_estimated_delivery    0
diff                           0
customer_unique_id             0
dtype: int64

In [93]:
o_status_del

,order_id,customer_id,order_status,ts_order_purchase,ts_order_delivered_customer,ts_order_estimated_delivery,diff,customer_unique_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2018-10-02 10:56:00,2018-10-10 21:25:00,2018-10-18,15 days 13:04:00,7c396fd4830fd04220f754e42b4e5bff
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2019-07-24 20:41:00,2019-08-07 15:27:00,2019-08-13,19 days 03:19:00,af07308b275d755c9edb36a90c618231
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2019-08-08 08:38:00,2019-08-17 18:06:00,2019-09-04,26 days 15:22:00,3a653a41f6f9fc3d2a113cf8398680e8
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2018-11-18 19:28:00,2018-12-02 00:28:00,2018-12-15,26 days 04:32:00,7c142cf63193a1473d2e66489a9ae977
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2019-02-13 21:18:00,2019-02-16 18:17:00,2019-02-26,12 days 02:42:00,72632f0f9dd73dfee390c9b22eb56dd6
...,...,...,...,...,...,...,...,...
96472,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2018-03-09 09:54:00,2018-03-17 15:08:00,2018-03-28,18 days 14:06:00,6359f309b166b0196dbf7ad2ac62bb5a
96473,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2019-02-06 12:58:00,2019-02-28 17:37:00,2019-03-02,23 days 11:02:00,da62f9e57a76d978d02ab5362c509660
96474,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2018-08-27 14:46:00,2018-09-21 11:24:00,2018-09-27,30 days 09:14:00,737520a9aad80b3fbbdad19b66b37b30
96475,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2019-01-08 21:28:00,2019-01-25 23:32:00,2019-02-15,37 days 02:32:00,5097a5312c8b157bb7be58ae360ef43c


In [94]:
#to compute the tenure of each customer
last_purchase = o_status_del.groupby(['customer_unique_id'], as_index = False)['ts_order_purchase'].max()
first_purchase = o_status_del.groupby(['customer_unique_id'], as_index = False)['ts_order_purchase'].min()

In [95]:
tenure = pd.merge(last_purchase, first_purchase, on='customer_unique_id', how='inner')
tenure['tenure'] = tenure['ts_order_purchase_x'] - tenure['ts_order_purchase_y']

In [96]:
tenure.describe()

,tenure
count,93350
mean,2 days 15:29:23.233208355
std,25 days 00:13:09.912216646
min,0 days 00:00:00
25%,0 days 00:00:00
50%,0 days 00:00:00
75%,0 days 00:00:00
max,633 days 02:02:00


In [97]:
## Just account for the max delivery difference days 
max_diff = o_status_del.groupby(['customer_unique_id'], as_index = False)['diff'].max()
max_diff.columns = max_diff.columns.str.replace('diff', 'new_diff')

In [98]:
a = pd.merge(tenure[['tenure','customer_unique_id']], max_diff, on='customer_unique_id', how='inner')

In [99]:
o_status_del = pd.merge(o_status_del, a, on='customer_unique_id', how='left')

In [100]:
o_status_del

,order_id,customer_id,order_status,ts_order_purchase,ts_order_delivered_customer,ts_order_estimated_delivery,diff,customer_unique_id,tenure,new_diff
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2018-10-02 10:56:00,2018-10-10 21:25:00,2018-10-18,15 days 13:04:00,7c396fd4830fd04220f754e42b4e5bff,27 days 23:30:00,15 days 13:04:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2019-07-24 20:41:00,2019-08-07 15:27:00,2019-08-13,19 days 03:19:00,af07308b275d755c9edb36a90c618231,0 days 00:00:00,19 days 03:19:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2019-08-08 08:38:00,2019-08-17 18:06:00,2019-09-04,26 days 15:22:00,3a653a41f6f9fc3d2a113cf8398680e8,0 days 00:00:00,26 days 15:22:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2018-11-18 19:28:00,2018-12-02 00:28:00,2018-12-15,26 days 04:32:00,7c142cf63193a1473d2e66489a9ae977,0 days 00:00:00,26 days 04:32:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2019-02-13 21:18:00,2019-02-16 18:17:00,2019-02-26,12 days 02:42:00,72632f0f9dd73dfee390c9b22eb56dd6,0 days 00:00:00,12 days 02:42:00
...,...,...,...,...,...,...,...,...,...,...
96472,9c5dedf39a927c1b2549525ed64a053c,39bd1228ee8140590ac3aca26f2dfe00,delivered,2018-03-09 09:54:00,2018-03-17 15:08:00,2018-03-28,18 days 14:06:00,6359f309b166b0196dbf7ad2ac62bb5a,0 days 00:00:00,18 days 14:06:00
96473,63943bddc261676b46f01ca7ac2f7bd8,1fca14ff2861355f6e5f14306ff977a7,delivered,2019-02-06 12:58:00,2019-02-28 17:37:00,2019-03-02,23 days 11:02:00,da62f9e57a76d978d02ab5362c509660,0 days 00:00:00,23 days 11:02:00
96474,83c1379a015df1e13d02aae0204711ab,1aa71eb042121263aafbe80c1b562c9c,delivered,2018-08-27 14:46:00,2018-09-21 11:24:00,2018-09-27,30 days 09:14:00,737520a9aad80b3fbbdad19b66b37b30,0 days 00:00:00,30 days 09:14:00
96475,11c177c8e97725db2631073c19f07b62,b331b74b18dc79bcdf6532d51e1637c1,delivered,2019-01-08 21:28:00,2019-01-25 23:32:00,2019-02-15,37 days 02:32:00,5097a5312c8b157bb7be58ae360ef43c,0 days 00:00:00,37 days 02:32:00


In [101]:
customers.columns , o_payments1.columns, o_product.columns , o_item.columns , o_status_del.columns, p_reviews.columns

(Index(['customer_id', 'customer_unique_id', 'customer_autonomous_community',
        'customer_city', 'cac_count', 'cc_count', 'order_id', 'order_count'],
       dtype='object'),
 Index(['customer_unique_id', 'payment_installments_quantity', 'payment_method',
        'transaction_value', 'payment_method_sequence_id'],
       dtype='object'),
 Index(['product_id', 'order_id', 'og_category', 'new_category',
        'customer_unique_id', 'og_cat_combined', 'new_cat_combined'],
       dtype='object'),
 Index(['product_id', 'order_id', 'order_item_count', 'product_id_combined',
        'tot_price', 'tot_shipping_cost', 'total', 'product_count',
        'customer_unique_id', 'tot_spending', 'max_shipping',
        'tot_order_item_count', 'tot_shipping'],
       dtype='object'),
 Index(['order_id', 'customer_id', 'order_status', 'ts_order_purchase',
        'ts_order_delivered_customer', 'ts_order_estimated_delivery', 'diff',
        'customer_unique_id', 'tenure', 'new_diff'],
       dtype=

In [102]:
customers.shape , o_payments1.shape, o_product.shape , o_item.shape , o_status_del.shape, p_reviews.shape

((96477, 8), (93349, 5), (108661, 7), (96470, 13), (96477, 10), (98699, 6))

In [103]:
df = pd.merge(customers, o_payments1, on='customer_unique_id', how='inner')
df1 = pd.merge(o_product, o_item, on=['customer_unique_id','order_id','product_id'], how='inner')
df2 = pd.merge(p_reviews, o_status_del, on=['customer_unique_id','order_id'], how='inner')

In [104]:
df.columns, df1.columns, df2.columns 

(Index(['customer_id', 'customer_unique_id', 'customer_autonomous_community',
        'customer_city', 'cac_count', 'cc_count', 'order_id', 'order_count',
        'payment_installments_quantity', 'payment_method', 'transaction_value',
        'payment_method_sequence_id'],
       dtype='object'),
 Index(['product_id', 'order_id', 'og_category', 'new_category',
        'customer_unique_id', 'og_cat_combined', 'new_cat_combined',
        'order_item_count', 'product_id_combined', 'tot_price',
        'tot_shipping_cost', 'total', 'product_count', 'tot_spending',
        'max_shipping', 'tot_order_item_count', 'tot_shipping'],
       dtype='object'),
 Index(['order_id', 'product_id', 'review_score', 'review_date',
        'customer_unique_id', 'review_count', 'customer_id', 'order_status',
        'ts_order_purchase', 'ts_order_delivered_customer',
        'ts_order_estimated_delivery', 'diff', 'tenure', 'new_diff'],
       dtype='object'))

In [105]:
df3 = pd.merge(df, df1, on=['customer_unique_id','order_id'], how='inner')
df4 = pd.merge(df2, df3, on=['customer_unique_id','order_id','product_id','customer_id'], how='inner')

In [106]:
df4.isna().sum()

order_id                         0
product_id                       0
review_score                     0
review_date                      0
customer_unique_id               0
review_count                     0
customer_id                      0
order_status                     0
ts_order_purchase                0
ts_order_delivered_customer      0
ts_order_estimated_delivery      0
diff                             0
tenure                           0
new_diff                         0
customer_autonomous_community    0
customer_city                    0
cac_count                        0
cc_count                         0
order_count                      0
payment_installments_quantity    0
payment_method                   0
transaction_value                0
payment_method_sequence_id       0
og_category                      0
new_category                     0
og_cat_combined                  0
new_cat_combined                 0
order_item_count                 0
product_id_combined 

In [107]:
df4.order_count.value_counts()

1     96665
2      5626
3       610
4       141
5        47
6        37
7        25
15       14
9        10
Name: order_count, dtype: int64

In [108]:
df4[df4['order_count'] == 9]

,order_id,product_id,review_score,review_date,customer_unique_id,review_count,customer_id,order_status,ts_order_purchase,ts_order_delivered_customer,ts_order_estimated_delivery,diff,tenure,new_diff,customer_autonomous_community,customer_city,cac_count,cc_count,order_count,payment_installments_quantity,payment_method,transaction_value,payment_method_sequence_id,og_category,new_category,og_cat_combined,new_cat_combined,order_item_count,product_id_combined,tot_price,tot_shipping_cost,total,product_count,tot_spending,max_shipping,tot_order_item_count,tot_shipping
7462,1124c329070977fbd414f046bba149d7,689a9f33ae479ec0d9f68a9b6b0cfcbf,1,20/12/2018 21:35,3e43e6105506432c953e165fb2acf44c,13,b70f8552b91ef49129519206966e2472,delivered,2018-12-01 22:37:00,2018-12-19 19:15:00,2019-01-02,31 days 01:23:00,161 days 23:43:00,38 days 14:30:00,andalucia,cadiz,1,1,9,10.0,credit_card,1172.66,1.0,furniture,home & kitchen,"furniture,kitchen & dining,home accessories,to...","home & kitchen,computers & phones",1,689a9f33ae479ec0d9f68a9b6b0cfcbf,50.00,15.10,65.10,1,1172.67,31.13,14,171.82
7463,70863e8ef99613bbc8f854807d187ea7,d296d7996d75ec35fcfc75b3a06cae63,5,02/03/2019 12:26,3e43e6105506432c953e165fb2acf44c,13,2bbd32d4ef14893d2d8c1a0df08403cf,delivered,2019-02-12 10:12:00,2019-02-27 22:09:00,2019-03-06,21 days 13:48:00,161 days 23:43:00,38 days 14:30:00,andalucia,cadiz,1,1,9,10.0,credit_card,1172.66,1.0,kitchen & dining,home & kitchen,"furniture,kitchen & dining,home accessories,to...","home & kitchen,computers & phones",2,"ceeb8b2d571b23399910f1b83980c973,d296d7996d75e...",106.00,24.58,130.58,2,1172.67,31.13,14,171.82
7464,826b47e4cd7bba4e4c6fa5485f898b74,446607005fbc439a9439caed96867e26,5,20/12/2018 21:35,3e43e6105506432c953e165fb2acf44c,13,e68e6423401e85c138229b23d4bf4761,delivered,2018-12-01 22:37:00,2018-12-06 19:09:00,2018-12-27,25 days 01:23:00,161 days 23:43:00,38 days 14:30:00,andalucia,cadiz,1,1,9,10.0,credit_card,1172.66,1.0,furniture,home & kitchen,"furniture,kitchen & dining,home accessories,to...","home & kitchen,computers & phones",2,"82254d93b897fde054504b15f8fe923c,446607005fbc4...",336.90,27.28,364.18,2,1172.67,31.13,14,171.82
7465,caf10b4f81696bae84a2448c0767b3b0,f582884ed8bb76a4b6317b8c9e966d98,5,18/01/2019 00:16,3e43e6105506432c953e165fb2acf44c,13,b63c9cfc5dcb06ed307ca95957fa216c,delivered,2019-01-11 10:56:00,2019-01-16 19:07:00,2019-01-26,14 days 13:04:00,161 days 23:43:00,38 days 14:30:00,andalucia,cadiz,1,1,9,10.0,credit_card,1172.66,1.0,home accessories,home & kitchen,"furniture,kitchen & dining,home accessories,to...","home & kitchen,computers & phones",1,f582884ed8bb76a4b6317b8c9e966d98,119.00,13.95,132.95,1,1172.67,31.13,14,171.82
7466,e23ad32d5c15d9ad308baddc6262afea,957f6d6b55b78746f89e976cba785440,5,28/12/2018 20:04,3e43e6105506432c953e165fb2acf44c,13,a4413edae691fd0d28bd04b8a38b259d,delivered,2018-12-01 09:30:00,2018-12-27 17:22:00,2019-01-09,38 days 14:30:00,161 days 23:43:00,38 days 14:30:00,andalucia,cadiz,1,1,9,10.0,credit_card,1172.66,1.0,furniture,home & kitchen,"furniture,kitchen & dining,home accessories,to...","home & kitchen,computers & phones",1,957f6d6b55b78746f89e976cba785440,97.47,17.93,115.40,1,1172.67,31.13,14,171.82
7467,ed0b6f8c8617370e53735a5d963bb5b6,7b49e585ac8c80b14c9b2815d4db465f,4,26/09/2018 01:34,3e43e6105506432c953e165fb2acf44c,13,06ec7aed3161859018db02f695b6600d,delivered,2018-09-18 18:53:00,2018-09-22 20:56:00,2018-09-28,9 days 05:07:00,161 days 23:43:00,38 days 14:30:00,andalucia,cadiz,1,1,9,10.0,credit_card,1172.66,1.0,kitchen & dining,home & kitchen,"furniture,kitchen & dining,home accessories,to...","home & kitchen,computers & phones",4,"1269d60fc5efbcf03e7eea49213091c6,55363f487ca32...",67.60,31.13,98.73,3,1172.67,31.13,14,171.82
7468,ed0b6f8c8617370e53735a5d963bb5b6,7b49e585ac8c80b14c9b2815d4db465f,4,26/09/2018 01:34,3e43e6105506432c953e165fb2acf44c,13,06ec7aed3161859018db02f695b6600d,delivered,2018-09-18 18:53:00,2018-09-22 20:56:00,2018-09-28,9 days 05:07:00,161 days 23:43:00,38 day

In [109]:
def set_group(row):
    if row["payment_installments_quantity"] > 1.0 :
        return 1
    else:
        return 0

In [110]:
df4 = df4.assign(installment_payment=df4.apply(set_group, axis=1))

In [111]:
df4['review_date'] = pd.to_datetime(df4['review_date'], format='%d/%m/%Y %H:%M')

In [112]:
df4.columns

Index(['order_id', 'product_id', 'review_score', 'review_date',
       'customer_unique_id', 'review_count', 'customer_id', 'order_status',
       'ts_order_purchase', 'ts_order_delivered_customer',
       'ts_order_estimated_delivery', 'diff', 'tenure', 'new_diff',
       'customer_autonomous_community', 'customer_city', 'cac_count',
       'cc_count', 'order_count', 'payment_installments_quantity',
       'payment_method', 'transaction_value', 'payment_method_sequence_id',
       'og_category', 'new_category', 'og_cat_combined', 'new_cat_combined',
       'order_item_count', 'product_id_combined', 'tot_price',
       'tot_shipping_cost', 'total', 'product_count', 'tot_spending',
       'max_shipping', 'tot_order_item_count', 'tot_shipping',
       'installment_payment'],
      dtype='object')

In [113]:
df4['delivered/reviewed'] = df4['review_date'] - df4['ts_order_delivered_customer']

In [114]:
df4['rev_count/tot_items'] = df4['review_count'] / df4['tot_order_item_count']

In [115]:
dataset = df4[['customer_unique_id','tenure', 'tot_spending','payment_method','max_shipping','tot_order_item_count','installment_payment','order_count', 'new_diff', 'og_cat_combined', 'new_cat_combined', 'customer_autonomous_community', 
               'order_item_count','product_count','tot_order_item_count','customer_city','review_count','delivered/reviewed','rev_count/tot_items','tot_shipping']]

In [116]:
dataset

,customer_unique_id,tenure,tot_spending,payment_method,max_shipping,tot_order_item_count,installment_payment,order_count,new_diff,og_cat_combined,new_cat_combined,customer_autonomous_community,order_item_count,product_count,tot_order_item_count,customer_city,review_count,delivered/reviewed,rev_count/tot_items,tot_shipping
0,871766c5855e863f6eccc05f988b23cb,0 days 00:00:00,72.19,credit_card,13.29,1,1,1,15 days 15:01:00,comics,books,cataluna,1,1,1,tarragona,1,1 days 11:14:00,1.0,13.29
1,eb28e67c4c0b83846050ddfb8a35d051,145 days 05:19:00,284.56,credit_card,19.93,2,1,2,18 days 13:07:00,"pet supplies,computer accessories","pets,computers & phones",andalucia,1,1,2,cadiz,2,2 days 19:30:00,1.0,31.78
2,eb28e67c4c0b83846050ddfb8a35d051,145 days 05:19:00,284.56,credit_card,19.93,2,1,2,18 days 13:07:00,"pet supplies,computer accessories","pets,computers & phones",andalucia,1,1,2,cadiz,2,3 days 13:43:00,1.0,31.78
3,af861d436cfc08b2c2ddefd0ba074622,0 days 00:00:00,25.78,credit_card,12.79,1,1,1,11 days 14:00:00,fragrance,beauty & personal care,andalucia,1,1,1,cadiz,1,1 days 03:07:00,1.0,12.79
4,64b576fb70d441e8f1b2d7d446e483c5,0 days 00:00:00,218.04,credit_card,18.14,1,1,1,40 days 10:03:00,lawn garden,garden,andalucia,1,1,1,cordoba,1,1 days 18:12:00,1.0,18.14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103170,ecc3d4eb9b17d2f0865d21f2abecc51c,0 days 00:00:00,139.88,credit_card,20.03,1,1,1,33 days 14:33:00,tools home improvement,home & kitchen,comunidad de madrid,1,1,1,getafe,1,3 days 16:50:00,1.0,20.03
103171,0da9fe112eae0c74d3ba1fe16de0988b,0 days 00:00:00,386.53,cash,36.53,1,0,1,17 days 13:34:00,computer accessories,computers & phones,galicia,1,1,1,coruna,1,1 days 12:54:00,1.0,36.53
103172,cd79b407828f02fdbba457111c38e4c4,0 days 00:00:00,116.85,credit_card,16.95,1,1,1,17 days 06:53:00,sport outdoors,sport,comunidad de madrid,1,1,1,madrid,1,1 days 09:11:00,1.0,16.95
103173,eb803377c9315b564bdedad672039306,0 days 00:00:00,64.71,credit_card,8.72,1,1,1,10 days 00:58:00,computer accessories,computers & phones,andalucia,1,1,1,sevilla,1,1 days 14:25:00,1.0,8.72


In [117]:
df4[df4['delivered/reviewed'] < "0 days"]

,order_id,product_id,review_score,review_date,customer_unique_id,review_count,customer_id,order_status,ts_order_purchase,ts_order_delivered_customer,ts_order_estimated_delivery,diff,tenure,new_diff,customer_autonomous_community,customer_city,cac_count,cc_count,order_count,payment_installments_quantity,payment_method,transaction_value,payment_method_sequence_id,og_category,new_category,og_cat_combined,new_cat_combined,order_item_count,product_id_combined,tot_price,tot_shipping_cost,total,product_count,tot_spending,max_shipping,tot_order_item_count,tot_shipping,installment_payment,delivered/reviewed,rev_count/tot_items
37,0017afd5076e074a48f1f1a4c7bac9c5,fe59a1e006df3ac42bf0ceb876d70969,5,2018-05-21 11:30:00,98758d88bf4b8eef1372ddee45d63178,2,8085a9af46f619bc25966f151a362b0d,delivered,2018-04-06 22:16:00,2018-05-23 08:32:00,2018-05-19,42 days 01:44:00,163 days 08:28:00,42 days 01:44:00,pais vasco,vitoria,1,1,2,10.0,credit_card,997.96,1.0,computer accessories,computers & phones,"computer accessories,comics","computers & phones,books",1,fe59a1e006df3ac42bf0ceb876d70969,809.10,44.29,853.39,1,997.96,44.29,2,78.86,1,-2 days +02:58:00,1.000000
45,001c85b5f68d2be0cb0797afc9e8ce9a,84f456958365164420cfc80fbe4c7fab,5,2018-12-20 21:03:00,55a269f324455e78349e7b9b7e7e5911,1,48ed31e735f1c420ed6ca3637b7c744d,delivered,2018-11-24 19:19:00,2018-12-22 18:37:00,2018-12-14,19 days 04:41:00,0 days 00:00:00,19 days 04:41:00,comunidad de madrid,madrid,1,1,1,4.0,credit_card,112.71,1.0,kitchen & dining,home & kitchen,kitchen & dining,home & kitchen,1,84f456958365164420cfc80fbe4c7fab,99.00,13.71,112.71,1,112.71,13.71,1,13.71,1,-2 days +02:26:00,1.000000
79,0030d783f979fbc5981e75613b057344,ae27a5524edb2c8dc4656c670f458fb7,4,2018-12-19 20:26:00,199888480790cc4f6135f3a3e11af83e,1,84ddc138522822dfb51b603c2f466a18,delivered,2018-11-24 20:03:00,2019-01-06 00:25:00,2018-12-15,20 days 03:57:00,0 days 00:00:00,20 days 03:57:00,cataluna,barcelona,1,1,1,7.0,credit_card,78.27,1.0,home accessories,home & kitchen,home accessories,home & kitchen,1,ae27a5524edb2c8dc4656c670f458fb7,60.60,17.67,78.27,1,78.27,17.67,1,17.67,1,-18 days +20:01:00,1.000000
82,0032d07457ae9c806c79368d7d9ce96b,08279c494018541f71443c07d77560f8,3,2019-04-19 10:53:00,98e71752819916789567071c52c4239e,1,d356c20816dc75a309628b5c17d89468,delivered,2019-03-10 18:53:00,2019-04-29 21:08:00,2019-04-17,37 days 05:07:00,0 days 00:00:00,37 days 05:07:00,cataluna,barcelona,1,1,1,2.0,credit_card,186.19,1.0,home accessories,home & kitchen,home accessories,home & kitchen,1,08279c494018541f71443c07d77560f8,159.00,27.19,186.19,1,186.19,27.19,1,27.19,1,-11 days +13:45:00,1.000000
85,00335b686d693c7d72deeb12f8e89227,87b08e712cc4c9fe70984c5a24b29e2f,1,2018-08-02 22:09:00,22b12c9c5bc6ba994b2423d87d3bb88a,1,d96e5c4400413a11fa8c9fd54be4a20b,delivered,2018-07-17 21:25:00,2018-09-12 20:58:00,2018-08-11,24 days 02:35:00,0 days 00:00:00,24 days 02:35:00,castilla-la mancha,toledo,1,1,1,1.0,cash,80.79,1.0,toys games,kids & toys,toys games,kids & toys,1,87b08e712cc4c9fe70984c5a24b29e2f,63.90,16.89,80.79,1,80.79,16.89,1,16.89,0,-41 days +01:11:00,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103118,ffdd178654127ccf34a439e8704fecd9,16b4473e98422039c388f144a0b16f55,5,2019-04-06 15:39:00,57e2b4ad2d77ceb7034ddaf45564600d,1,33d0814c9eff200629eb49330ce5c6a3,delivered,2019-03-07 14:07:00,2019-04-07 13:16:00,2019-04-04,27 days 09:53:00,0 days 00:00:00,27 days 09:53:00,pais vasco,vitoria,1,1,1,5.0,credit_card,108.61,1.0,beauty & personal care,beauty & personal care,beauty & personal care,beauty & personal care,1,16b4473e98422039c388f144a0b16f55,69.90,38.71,108.61,1,108.61,38.71,1,38.71,1,-1 days +02:23:00,1.000000
103143,ffecd5a79a0084f6a592288c67e3c298,50fd2b788dc166edd20512370dac54df,5,2019-03-24 15:21:00,a969e99358a58bed2f4073fd04165a2b,1,8afa5a415d65a08e3a68b1d7f9ddd8a5,delivered,2019-02-27 20:00:00,2019-03-2

In [118]:
df4[df4['customer_unique_id'] == "98758d88bf4b8eef1372ddee45d63178"]

,order_id,product_id,review_score,review_date,customer_unique_id,review_count,customer_id,order_status,ts_order_purchase,ts_order_delivered_customer,ts_order_estimated_delivery,diff,tenure,new_diff,customer_autonomous_community,customer_city,cac_count,cc_count,order_count,payment_installments_quantity,payment_method,transaction_value,payment_method_sequence_id,og_category,new_category,og_cat_combined,new_cat_combined,order_item_count,product_id_combined,tot_price,tot_shipping_cost,total,product_count,tot_spending,max_shipping,tot_order_item_count,tot_shipping,installment_payment,delivered/reviewed,rev_count/tot_items
37,0017afd5076e074a48f1f1a4c7bac9c5,fe59a1e006df3ac42bf0ceb876d70969,5,2018-05-21 11:30:00,98758d88bf4b8eef1372ddee45d63178,2,8085a9af46f619bc25966f151a362b0d,delivered,2018-04-06 22:16:00,2018-05-23 08:32:00,2018-05-19,42 days 01:44:00,163 days 08:28:00,42 days 01:44:00,pais vasco,vitoria,1,1,2,10.0,credit_card,997.96,1.0,computer accessories,computers & phones,"computer accessories,comics","computers & phones,books",1,fe59a1e006df3ac42bf0ceb876d70969,809.1,44.29,853.39,1,997.96,44.29,2,78.86,1,-2 days +02:58:00,1.0
38,b6f41799a58be93e516fb6992e0d4b47,cbe952607a2215211502095c631ea179,5,2018-10-19 15:44:00,98758d88bf4b8eef1372ddee45d63178,2,58284e22c11c1cd2a52d4f25630a1ed9,delivered,2018-09-17 06:44:00,2018-10-16 18:13:00,2018-10-17,29 days 17:16:00,163 days 08:28:00,42 days 01:44:00,pais vasco,vitoria,1,1,2,10.0,credit_card,997.96,1.0,comics,books,"computer accessories,comics","computers & phones,books",1,cbe952607a2215211502095c631ea179,110.0,34.57,144.57,1,997.96,44.29,2,78.86,1,2 days 21:31:00,1.0


In [119]:
df4[df4['delivered/reviewed'] < "0 days"]['customer_unique_id'].duplicated().sum()

387

In [120]:
df4['delivered/reviewed'] = df4['delivered/reviewed'].dt.days
df4['tenure'] = df4['tenure'].dt.days
df4['new_diff'] = df4['new_diff'].dt.days

In [121]:
df5 = df4.groupby(['customer_unique_id'], as_index = False).agg({'delivered/reviewed': 'mean',
                                                              'rev_count/tot_items': 'mean'})
df5 = df5.rename(columns={'delivered/reviewed':'del/rev_mean', 'rev_count/tot_items': 'rev_count/tot_items_mean'})
df4 = pd.merge(df4, df5, on='customer_unique_id', how='left')

In [124]:
df4

,order_id,product_id,review_score,review_date,customer_unique_id,review_count,customer_id,order_status,ts_order_purchase,ts_order_delivered_customer,ts_order_estimated_delivery,diff,tenure,new_diff,customer_autonomous_community,customer_city,cac_count,cc_count,order_count,payment_installments_quantity,payment_method,transaction_value,payment_method_sequence_id,og_category,new_category,og_cat_combined,new_cat_combined,order_item_count,product_id_combined,tot_price,tot_shipping_cost,total,product_count,tot_spending,max_shipping,tot_order_item_count,tot_shipping,installment_payment,delivered/reviewed,rev_count/tot_items,del/rev_mean,rev_count/tot_items_mean
0,00010242fe8c5a6d1ba2dd792cb16214,4244733e06e7ecb4970a6e2683c13e61,4,2018-09-22 10:57:00,871766c5855e863f6eccc05f988b23cb,1,3ce436f183e68e07877b285a838db11a,delivered,2018-09-13 08:59:00,2018-09-20 23:43:00,2018-09-29,15 days 15:01:00,0,15,cataluna,tarragona,1,1,1,2.0,credit_card,72.19,1.0,comics,books,comics,books,1,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,72.19,1,72.19,13.29,1,13.29,1,1,1.0,1.0,1.0
1,00018f77f2f0320c557190d7a144bdd3,e5f2d52b802189ee658865ca93d83a8f,5,2018-05-15 11:34:00,eb28e67c4c0b83846050ddfb8a35d051,2,f6dd3ec061db4e3987629fe6b26e5cce,delivered,2018-04-26 10:53:00,2018-05-12 16:04:00,2018-05-15,18 days 13:07:00,145,18,andalucia,cadiz,1,1,2,3.0,credit_card,284.56,1.0,pet supplies,pets,"pet supplies,computer accessories","pets,computers & phones",1,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,259.83,1,284.56,19.93,2,31.78,1,2,1.0,2.5,1.0
2,c105be01f21c9df6a2579f59b95cbcd9,d2f5484cbffe4ca766301b21ab9246dd,1,2018-09-29 11:42:00,eb28e67c4c0b83846050ddfb8a35d051,2,02288a47f696f3054b9ef1d5f8e73973,delivered,2018-09-18 16:12:00,2018-09-25 21:59:00,2018-10-04,15 days 07:48:00,145,18,andalucia,cadiz,1,1,2,3.0,credit_card,284.56,1.0,computer accessories,computers & phones,"pet supplies,computer accessories","pets,computers & phones",1,d2f5484cbffe4ca766301b21ab9246dd,12.88,11.85,24.73,1,284.56,19.93,2,31.78,1,3,1.0,2.5,1.0
3,00024acbcdf0a6daa1e931b038114c75,7634da152a4610f1595efa32f14722fc,5,2019-08-15 16:39:00,af861d436cfc08b2c2ddefd0ba074622,1,d4eb9395c8c0431ee92fce09860c5a06,delivered,2019-08-08 10:00:00,2019-08-14 13:32:00,2019-08-20,11 days 14:00:00,0,11,andalucia,cadiz,1,1,1,2.0,credit_card,25.78,1.0,fragrance,beauty & personal care,fragrance,beauty & personal care,1,7634da152a4610f1595efa32f14722fc,12.99,12.79,25.78,1,25.78,12.79,1,12.79,1,1,1.0,1.0,1.0
4,00042b26cf59d7ce69dfabb4e55b4fd9,ac6c3623068f30de03045865e4e10089,5,2018-03-03 10:54:00,64b576fb70d441e8f1b2d7d446e483c5,1,58dbd0b2d70206bf40e62cd34e84d795,delivered,2018-02-04 13:57:00,2018-03-01 16:42:00,2018-03-17,40 days 10:03:00,0,40,andalucia,cordoba,1,1,1,3.0,credit_card,218.04,1.0,lawn garden,garden,lawn garden,garden,1,ac6c3623068f30de03045865e4e10089,199.90,18.14,218.04,1,218.04,18.14,1,18.14,1,1,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103170,fffbee3b5462987e66fb49b1c5411df2,6f0169f259bb0ff432bfff7d829b9946,3,2019-07-09 10:41:00,ecc3d4eb9b17d2f0865d21f2abecc51c,1,11a0e041ea6e7e21856d2689b64e7f3a,delivered,2019-06-19 09:27:00,2019-07-05 17:51:00,2019-07-23,33 days 14:33:00,0,33,comunidad de madrid,getafe,1,1,1,3.0,credit_card,139.88,1.0,tools home improvement,home & kitchen,tools home improvement,home & kitchen,1,6f0169f259bb0ff432bfff7d829b9946,119.85,20.03,139.88,1,139.88,20.03,1,20.03,1,3,1.0,3.0,1.0
103171,fffcd46ef2263f404302a634eb57f7eb,32e07fd915822b0765e448c4dd74c828,5,2019-07-25 09:25:00,0da9fe112eae0c74d3ba1fe16de0988b,1,84c5d4fbaf120aae381fad077416eaa0,delivered,2019-07-14 10:26:00,2019-07-23 20:31:00,2019-08-01,17 days 13:34:00,0,17,galicia,coruna,1,1,1,1.0,cash,386.53,1.0,computer accessories,computers & phones,computer accessories,computers & phones,1,32e07fd915822b0765e448c4dd74c828,350.00,36.53,386.53,1,386.53,36.53,1,36.53,0,1,1.0,1.0,1

In [129]:
customer_unique_df = df4.drop_duplicates(['customer_unique_id'],keep= 'last')

In [130]:
customer_unique_df.isna().sum()

order_id                         0
product_id                       0
review_score                     0
review_date                      0
customer_unique_id               0
review_count                     0
customer_id                      0
order_status                     0
ts_order_purchase                0
ts_order_delivered_customer      0
ts_order_estimated_delivery      0
diff                             0
tenure                           0
new_diff                         0
customer_autonomous_community    0
customer_city                    0
cac_count                        0
cc_count                         0
order_count                      0
payment_installments_quantity    0
payment_method                   0
transaction_value                0
payment_method_sequence_id       0
og_category                      0
new_category                     0
og_cat_combined                  0
new_cat_combined                 0
order_item_count                 0
product_id_combined 

In [131]:
customer_unique_df.shape

(90751, 42)

In [132]:
#df4.to_csv('C:/Users/elfek/Desktop/NTT/clean/df4.csv')
#dataset.to_csv('C:/Users/elfek/Desktop/NTT/clean/dataset.csv')
customer_unique_df.to_csv('C:/Users/elfek/Desktop/NTT/clean/customer_unique_df.csv')